# Colorizing Australia

In this assignment, you will implement the backtracking search for colorizing the Australia on your own.

In [1]:
class CSP(object):
    def __init__(self):
        self.variables = []
        self.domains = {}
        self.factors = {}

    def add_variable(self, variable, domain):
        """
        Takes variable and its domain and adds them to the CSP
        :param variable: any type
        :param domain: list of possible values for teh variable
        """
        if variable in self.variables: raise ValueError('This CSP already contains variable', variable, ', please use another name or check the consistency of your code.' )
        if type(domain) is not list: raise ValueError('Domain should be a list.' )
        self.variables.append(variable)
        self.domains[variable] = domain

    def add_factor(self, variables, factor_function):
        """
        Takes variables and the potential for these variables and adds them to the CSP
        :param variables: set of variables
        :param potential: potential function taking values of the variables as input and returning 
        non-negative value of the potential  
        """

        if type(variables) not in (list, set): raise ValueError('Variables should be a list or a set.' )
        if type(variables) is list: variables = frozenset(variables)

        if variables in self.factors.keys():
            self.factors[variables].append(factor_function)
        else:
            self.factors[variables] = [factor_function]

Here is the code constructing CSP for you. Examine it carefuly, make sure you understand all the srtuctures used.

In [2]:
import copy
csp_Australia = CSP()
provinces = ['WA', 'NT', 'Q', 'NSW', 'V', 'SA', 'T']
neighbors = {
    'SA' : ['WA', 'NT', 'Q', 'NSW', 'V'],
    'NT' : ['WA', 'Q'],
    'NSW' : ['Q', 'V']
}

colors = ['red', 'blue', 'green']

def are_neighbors(a, b):
    return (a in neighbors and b in neighbors[a]) or (b in neighbors and a in neighbors[b])

for p in provinces:
    csp_Australia.add_variable(p, copy.deepcopy(colors))
for p1 in provinces:
    for p2 in provinces:
        if are_neighbors(p1, p2):
            
            # Neighbors cannot have the same color
            csp_Australia.add_factor([p1, p2], lambda x, y : x != y)

Here you will implement backtracking search with advanced forward checking and without propagation through a singleton domain for a good reason.

Backtracking search will work like the following: it takes partial assignment is a form of all possible values to the variables (initially - full domains of the CSP). Next, we will do assignments by eliminating everything except the assigned value from the variable's domain. This way, we already enforce Prop-1 by design. 

Here is scp_solver class where you should add your code.

In [3]:
class csp_solver():

    def solve(self, csp):
        self.csp = csp

        # Dictionary for storing all valid assignments. Note, unlike standard CSP when we
        # are interested only in one valid assignment, for this problem you need to
        # find all valid assignments
        self.valid_assignments = []

        # Number of all valid assignments

        # Number of times backtracking operation is called
        self.num_operations = 0

        self.backtrack(copy.deepcopy(csp.domains))

        if self.valid_assignments != []:
            print('Found %d optimal assignments in %d operations' % (len(self.valid_assignments), self.num_operations))
            for assignment in self.valid_assignments:
                print(assignment)
        else:
            print('No assignments was found.')
        

    def backtrack(self, partial_assignment):
        """

        2c.

        Here you will implement backtracking search with arc consistency (advanced forward checking) and without propagation
        through a singleton domain for a good reason.

        Backtracking search will work like the following: it takes partial assignment is a form of
        all possible values to the variables (initially - full domains of the CSP). Next, we will do assignments
        by eliminating everything except the assigned value from the variable's domain. This way, we already enforce
        Prop-1 by design.

        If the valid assignment is found, add it to self.valid_assignments in a form of (assignment:}

        :param partial_assignment: a dictionary containing partial assignment in a form {variable:list of possible values}
        """
        self.num_operations += 1
        
        assigned_var = self.choose_next_variable_min(partial_assignment)  # choose next variable with smallest domain size
        #assigned_var = self.choose_next_variable_max(partial_assignment)  # choose next variable with leargest domain size

        for col in partial_assignment[assigned_var]:

            temp_partial_assignment = copy.deepcopy(partial_assignment)
            temp_partial_assignment[assigned_var] = [col] # assign color to the assigned variable
            # Run forward_checking to eliminate domains from neighbors 
            temp_partial_assignment = self.forward_checking(assigned_var, temp_partial_assignment, self.csp.factors)
            
            if not self.consistent(temp_partial_assignment): # check for consistency
                continue
                
            if self.is_valid_assignment(temp_partial_assignment):
                self.valid_assignments.append(temp_partial_assignment)
                continue
                
            # need to continue backtracking, finished search for this variable -> choose next one
            self.backtrack(temp_partial_assignment) 


    def consistent(self, partial_assignment):
        """ Check if any domains is empty."""
        return all([value for value in partial_assignment.values()])

    
    def is_valid_assignment(self, assignment):
        """ Check if we found one final result."""
        check_all = sum([len(value) for value in assignment.values()])
        return check_all == len(assignment)

    
    def choose_next_variable_max(self, partial_assignment):
        """
        2a.
        :param partial_assignment: a dictionary containing partial assignment in a form {variable:list of values}
        :return: variable for partial assignment
        """
        list_length = [len(partial_assignment.get(str(value))) for value in partial_assignment.keys()]
        return list(partial_assignment.keys())[list_length.index(max(list_length))]

    
    def choose_next_variable_min(self, partial_assignment):
        """
        2d. Better Heuristic
        :param partial_assignment: a dictionary containing partial assignment in a form {variable:list of values}
        :return: variable for partial assignment
        """
        list_length = []
        values = []
        for value in partial_assignment.keys():
            length = len(partial_assignment.get(str(value)))
            if length > 1:
                list_length.append(length)
                values.append(value)

        return values[list_length.index(min(list_length))]


    def forward_checking(self, assigned_variable, partial_assignment, factors):
        """
        2b.
        Implements forward checking on steroids. Checks if any domain contains values inconsistent with current assignment
        and eliminate these variables from the domain. As a result of this domain reduction there could be another
        inconsistency in the domains. Eliminate them recursively by keeping track of the reduced domain and calling forward_checking
        as a recursion.

        This wild version of forward checking is called arc consistency and is one of the most efficient implementation of the
        forward checking idea for CSP.

        :param assigned_variable: recently assigned variable or the variable for which the domain has been reduced
        :param partial_assignment: a dictionary containing partial assignment in a form {variable:list of values}
        :param factors: a dictionary containing factoors of the CSP if the form of {frozenset(variables):list of constraint functions}
        :return: a dictionary of partial assignments with reduced domains
        """
        assigned_colour = partial_assignment[assigned_variable][0]   # coosed colour
        temp_partial_assignment = copy.deepcopy(partial_assignment)
        for var in partial_assignment.keys():
            if frozenset([assigned_variable, var]) in factors:  # check if neighbours
                try:
                    temp_partial_assignment[var].remove(assigned_colour)
                except ValueError:
                    continue
                    
                if len(temp_partial_assignment[var]) == 1:    # if variable has one domain left run again forward checking
                    temp_partial_assignment = self.forward_checking(var, temp_partial_assignment, factors)
                    
                elif len(temp_partial_assignment[var]) == 0:
                    return temp_partial_assignment

        return temp_partial_assignment

In [4]:
solver = csp_solver()
solver.solve(csp_Australia)

Found 18 optimal assignments in 10 operations
{'WA': ['red'], 'NT': ['blue'], 'Q': ['red'], 'NSW': ['blue'], 'V': ['red'], 'SA': ['green'], 'T': ['red']}
{'WA': ['red'], 'NT': ['blue'], 'Q': ['red'], 'NSW': ['blue'], 'V': ['red'], 'SA': ['green'], 'T': ['blue']}
{'WA': ['red'], 'NT': ['blue'], 'Q': ['red'], 'NSW': ['blue'], 'V': ['red'], 'SA': ['green'], 'T': ['green']}
{'WA': ['red'], 'NT': ['green'], 'Q': ['red'], 'NSW': ['green'], 'V': ['red'], 'SA': ['blue'], 'T': ['red']}
{'WA': ['red'], 'NT': ['green'], 'Q': ['red'], 'NSW': ['green'], 'V': ['red'], 'SA': ['blue'], 'T': ['blue']}
{'WA': ['red'], 'NT': ['green'], 'Q': ['red'], 'NSW': ['green'], 'V': ['red'], 'SA': ['blue'], 'T': ['green']}
{'WA': ['blue'], 'NT': ['red'], 'Q': ['blue'], 'NSW': ['red'], 'V': ['blue'], 'SA': ['green'], 'T': ['red']}
{'WA': ['blue'], 'NT': ['red'], 'Q': ['blue'], 'NSW': ['red'], 'V': ['blue'], 'SA': ['green'], 'T': ['blue']}
{'WA': ['blue'], 'NT': ['red'], 'Q': ['blue'], 'NSW': ['red'], 'V': ['blue'], 

# Better heuristics
Number of operations changes when we choose next variable by finding variable with minimum number of possible domains rather than with maximum.
I created two functions: choose_next_variable_min and choose_next_variable_max.
Backtrack with choose_next_variable_max works for 19 operations.
Backtrack with choose_next_variable_min works for 10 operations.

Choosing min instead of max changes situation as 'for loop' inside backtrack function is called for n times, where n is number of colors(domains) that variable has. The smaller is this number the less operations we need to do.
